# S3W9D2: 向量化原理 (Embedding & Latent Space)

**今日目标**：
理解如何将自然语言映射到高维数学空间，并掌握计算语义相似度（Semantic Similarity）的核心算法。

## 1. 理论知识讲解 (CS Principle)

在计算机科学中，Embedding 是将**离散变量 (Discrete Variables)**（如单词、句子）映射为**连续向量 (Continuous Vectors)** 的过程。

### 核心概念：

1. **Latent Space (潜在空间)**：
* 想象一个 768 维的坐标系。在这个空间里，语义相近的词，距离会更近。
* 经典例子：。这表明模型学会了“性别”和“皇室”这两个维度的特征。


2. **Dense Vector (稠密向量)**：
* One-Hot 编码是**稀疏**的（维度爆炸，绝大多数是 0）。
* Embedding 是**稠密**的（维度固定，如 768 或 1024，每一位都是非零浮点数）。


3. **Cosine Similarity (余弦相似度)**：
* 衡量两个向量在方向上的差异，而不是距离（Magnitude）。
* **公式**：


* 值域为 `[-1, 1]`。1 表示完全重合（语义相同），0 表示正交（不相关），-1 表示相反。

## 2. 代码实现 (Code Implementation)

我们将使用 BAAI（智源）发布的 `bge-small-zh` 模型，它是目前中文开源界性价比最高的模型之一。

### 2.1 加载模型与基本编码

In [1]:
import os 
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from sentence_transformers import SentenceTransformer
import numpy as np

# 1. 加载模型
# 'BAAI/bge-small-zh-v1.5' 会自动下载到本地缓存
print("Loading model...")
model = SentenceTransformer('BAAI/bge-small-zh-v1.5')

# 2. 准备测试文本
sentences = [
    "如何根据刑法判处盗窃罪？",          # Query
    "盗窃公私财物，数额较大的，处三年以下有期徒刑。",  # Positive Document (相关)
    "今天中午吃什么？",                   # Negative Document (完全无关)
    "诈骗公私财物，数额较大的，处三年以下有期徒刑。"   # Hard Negative (字面相似但语义不同)
]

# 3. 转化为向量 (Embedding)
embeddings = model.encode(sentences)

print(f"Shape of embeddings: {embeddings.shape}")
# 输出应该是 (4, 512) -> 4句话，每句512维

# 4. (可选) 验证一下相似度，确认模型工作正常
from sklearn.metrics.pairwise import cosine_similarity
query_vec = embeddings[0].reshape(1, -1)
sim_scores = cosine_similarity(query_vec, embeddings)

print("\n--- Similarity Scores ---")
for i, score in enumerate(sim_scores[0]):
    print(f"[{score:.4f}] {sentences[i]}")


Loading model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Shape of embeddings: (4, 512)

--- Similarity Scores ---
[1.0000] 如何根据刑法判处盗窃罪？
[0.6844] 盗窃公私财物，数额较大的，处三年以下有期徒刑。
[0.1875] 今天中午吃什么？
[0.5970] 诈骗公私财物，数额较大的，处三年以下有期徒刑。


### 2.2 计算相似度 (手动实现 vs 库函数)

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

# 获取 Query 的向量 (第一句话)
query_vec = embeddings[0].reshape(1, -1)

# 计算 Query 与所有句子的相似度
# cosine_similarity 接受 (n_samples_X, n_features) 矩阵
sim_scores = cosine_similarity(query_vec, embeddings)

print("Query:", sentences[0])
print("-" * 30)
for i, score in enumerate(sim_scores[0]):
    print(f"Score: {score:.4f} | Text: {sentences[i]}")

# 预期结果：
# 第2句 (盗窃) 分数最高 (接近 0.7-0.9)
# 第3句 (吃饭) 分数最低 (接近 0)
# 第4句 (诈骗) 分数居中 (因为结构相似，但核心实体不同)

Query: 如何根据刑法判处盗窃罪？
------------------------------
Score: 1.0000 | Text: 如何根据刑法判处盗窃罪？
Score: 0.6844 | Text: 盗窃公私财物，数额较大的，处三年以下有期徒刑。
Score: 0.1875 | Text: 今天中午吃什么？
Score: 0.5970 | Text: 诈骗公私财物，数额较大的，处三年以下有期徒刑。


## 3. 深度理论 (Deep Dive)

### 为什么用 Cosine Similarity 而不是欧氏距离 (Euclidean Distance)？

在 CS 和 NLP 面试中，这是一个高频考点。

1. **维度灾难 (Curse of Dimensionality)**：在高维空间中，欧氏距离对“距离”的区分度会变差。
2. **模长 (Magnitude) 的影响**：
    * 有些 embedding 模型（尤其是未归一化的），句子越长，模长可能越大。
    * 欧氏距离受模长影响大。
    * Cosine Similarity 只看**方向**。只要方向一致，我们就认为语义相似，这在 NLP 中更鲁棒。


3. **归一化 (Normalization)**：
    * 如果我们将所有向量都做 L2 归一化（即模长为 1），那么**欧氏距离和余弦相似度是单调相关的**（排序结果一致）。
    * $$||A-B||^2 = 2(1 - \cos(A, B))$$
    （当 $||A||=||B||=1$ 时）



## 4. 工程封装 (Engineering Encapsulation)

在实际工程中，模型加载非常耗时（几秒到几分钟），不能每次调用函数都加载一次。我们需要使用 **单例模式 (Singleton Pattern)** 或全局缓存。

**创建/编辑文件**：`src/rag/embedding.py`

```python
import os
from sentence_transformers import SentenceTransformer

# --- 关键修正 1: 全局注入镜像地址 ---
# 必须在 import SentenceTransformer 之前或初始化之前生效
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

class EmbeddingModel:
    """
    Embedding 模型单例封装
    """
    _instance = None
    _model = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(EmbeddingModel, cls).__new__(cls)
            
            # 模型名称
            model_name = os.getenv("EMBEDDING_MODEL_NAME", "BAAI/bge-small-zh-v1.5")
            print(f"Initializing Embedding Model: {model_name}...")
            
            try:
                # --- 关键修正 2: 优先尝试纯离线加载 ---
                # local_files_only=True 会强制库只看本地缓存，绝对不联网
                print("Attempting to load from local cache (Offline Mode)...")
                cls._model = SentenceTransformer(model_name, local_files_only=True)
                print("✅ Successfully loaded from local cache.")
                
            except Exception as e:
                print(f"⚠️ Local cache not found. Error: {e}")
                print("🌐 Attempting to download from HF Mirror...")
                
                # --- 关键修正 3: 降级方案 ---
                # 如果本地真没有，再通过镜像下载
                try:
                    cls._model = SentenceTransformer(model_name)
                    print("✅ Successfully downloaded and loaded.")
                except Exception as e2:
                    print(f"❌ Critical Error: Failed to load model. Check your network.")
                    raise e2

        return cls._instance

    def get_embedding(self, text: str) -> list:
        """获取单条文本的向量"""
        # normalize_embeddings=True 之后，点积等同于余弦相似度
        return self._model.encode(text, normalize_embeddings=True).tolist()

    def get_embeddings(self, texts: list) -> list:
        """批量获取文本向量"""
        return self._model.encode(texts, normalize_embeddings=True).tolist()

# 测试代码
if __name__ == "__main__":
    try:
        embedder = EmbeddingModel()
        vec = embedder.get_embedding("测试一下")
        print(f"Vector dimension: {len(vec)}")
    except Exception as e:
        print(f"\nFATAL: {e}")

```

## 5. 模拟面试 (Mock Interview)

**面试官**：*“如果用户搜索‘苹果’，你希望检索出水果还是手机？Embedding 模型怎么处理这种多义词（Polysemy）？”*

**参考回答范式**：

> “这取决于**上下文（Context）**。
> 传统的 Word2Vec 是**静态 Embedding**，‘苹果’这个词无论在哪里，向量都是固定的，确实无法区分。
> 但现在的 BERT/Transformer 类模型（如 BGE）生成的是**动态 Contextualized Embedding**。
> * 在句子‘苹果很好吃’中，‘苹果’的 Embedding 会聚合‘好吃’、‘水果’的特征。
> * 在句子‘苹果发布了新手机’中，‘苹果’的 Embedding 会聚合‘科技’、‘公司’的特征。
> 
> 
> 所以在 RAG 系统中，我们是对**整个句子或段落**进行 Embedding，而不是单个词，因此模型可以根据上下文准确区分语义。”

## 6. 今日 LeetCode 练习

* **题目**：[141. Linked List Cycle (环形链表)](https://leetcode.com/problems/linked-list-cycle/)
* **关联**：链表成环检测在工程中常用于检测死锁或循环依赖。
* **核心思想**：**快慢指针 (Floyd's Cycle-Finding Algorithm)**。
* 快指针一次走两步，慢指针一次走一步。
* 如果是环，快指针一定会在圈里“追上”慢指针（就像操场跑圈套圈）。



```python
class Solution:
    def hasCycle(self, head: Optional[ListNode]) -> bool:
        if not head or not head.next:
            return False
        
        slow = head
        fast = head.next
        
        while slow != fast:
            if not fast or not fast.next:
                return False
            slow = slow.next
            fast = fast.next.next
            
        return True

```


## 7. 今日任务总结

* [ ] 安装了 `sentence-transformers`。
* [ ] 实现了 `src/rag/embedding.py` 的单例封装。
* [ ] 深刻理解了 Cosine Similarity 的几何意义及数学公式。
* [ ] 完成了 LeetCode 141。

**下一步预告**：
有了向量，如果咱们有 100 万个文档，每次都要和 Query 算一次余弦相似度，CPU 会算到冒烟（ 复杂度）。
明天（Day 3），我们将引入 **向量数据库 (Vector DB)**，利用 **HNSW 索引** 将搜索复杂度降低到 。